In [4]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import wandb
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import r2_score
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.lines import Line2D
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import os
from helper import train_VAE 

In [5]:
X_train = pd.read_csv('../datasets/Admission/X_train.csv',index_col=0)  
X_test = pd.read_csv('../datasets/Admission/X_test.csv',index_col=0)  
y_train = pd.read_csv('../datasets/Admission/y_train.csv',index_col=0)  
y_test = pd.read_csv('../datasets/Admission/y_test.csv',index_col=0) 
X_train_with_confounder=X_train.drop(columns=['CGPA'])
X_test_with_confounder=X_test.drop(columns=['CGPA'])
print(X_train.shape)
print(X_train_with_confounder)
print(X_test.shape)
print(X_test_with_confounder.shape)
print(X_train.columns)
print(X_train_with_confounder.columns)
X_train_with_confounder

(400, 7)
     GRE Score  TOEFL Score  University Rating  SOP  LOR   Research
60         309          100                  2  3.0   3.0         0
298        330          114                  3  4.5   4.5         1
228        318          112                  3  4.0   3.5         0
141        332          118                  2  4.5   3.5         1
3          322          110                  3  3.5   2.5         1
..         ...          ...                ...  ...   ...       ...
381        319          105                  3  3.0   3.5         1
37         300          105                  1  1.0   2.0         0
153        324          105                  3  3.0   4.0         0
396        325          107                  3  3.0   3.5         1
101        312          105                  2  2.5   3.0         0

[400 rows x 6 columns]
(100, 7)
(100, 6)
Index(['GRE Score', 'TOEFL Score', 'University Rating', 'SOP', 'LOR ', 'CGPA',
       'Research'],
      dtype='object')
Index(['GRE 

,GRE Score,TOEFL Score,University Rating,SOP,LOR,Research
60,309,100,2,3.0,3.0,0
298,330,114,3,4.5,4.5,1
228,318,112,3,4.0,3.5,0
141,332,118,2,4.5,3.5,1
3,322,110,3,3.5,2.5,1
...,...,...,...,...,...,...
381,319,105,3,3.0,3.5,1
37,300,105,1,1.0,2.0,0
153,324,105,3,3.0,4.0,0
396,325,107,3,3.0,3.5,1


In [6]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_with_confounder)
X_test_scaled = scaler.transform(X_test_with_confounder)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
import itertools

# Define the ranges for your parameters
x_dim_values = [X_train_with_confounder.shape[1]]
z_dim_values = [3,4]
zz_dim_values = [1]
encoder_z_layers_values = [[10,10,10],[64,64,64]]
encoder_zz_layers_values = [[10,10]]
#decoder_layers_values = [[14,X_train_with_confounder.shape[1]],[32, 32,X_train_with_confounder.shape[1]],[128,128,128,X_train_with_confounder.shape[1]]]
fc_z_to_y_layers_values = [[12,12,1],[32,32,32,1]]

# Get all combinations
configurations = list(itertools.product(x_dim_values, z_dim_values, zz_dim_values, encoder_z_layers_values, encoder_zz_layers_values, fc_z_to_y_layers_values))

# Convert to a list of dictionaries
config_dicts = [{'x_dim': x_dim, 'z_dim': z_dim, 'zz_dim': zz_dim, 'encoder_z_layers': encoder_z_layers, 'encoder_zz_layers': encoder_zz_layers, 'decoder_layers': encoder_z_layers+ [x_dim], 'fc_z_to_y_layers': fc_z_to_y_layers} for x_dim, z_dim, zz_dim, encoder_z_layers, encoder_zz_layers, fc_z_to_y_layers in configurations]
print(config_dicts)

[{'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 1

In [8]:
# Define betas
BETAS=[
    #{'b1': 6, 'b2': 1, 'b3': 1, 'b4': 6,'b5':1},
    #{'b1': 1, 'b2': 1, 'b3': 1, 'b4': 1000,'b5':1},
    #{'b1': 100, 'b2': 1, 'b3': 100, 'b4': 100,'b5':1},
    #{'b1': 100,'b2': 1, 'b3': 1, 'b4': 10,'b5':1},  
    #{'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000,'b5':1},
    {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000,'b5':1}
]
import itertools

# Define the ranges for your parameters
lr_values = [1e-3]
batch_size_values = [16,32]
num_epochs_values = [600,800,1000]

# Get all combinations
configs = list(itertools.product(config_dicts, BETAS, lr_values, batch_size_values, num_epochs_values))

# Convert to a list of dictionaries
configs = [{'config': config, 'beta': beta, 'lr': lr, 'batch_size': bs, 'num_epochs': epochs} for config, beta, lr, bs, epochs in configs]
print(len(configs))

48


In [6]:
#train_VAE(X_train,X_test,X_test_original,y_train,y_test,configs,path,save_pictures=False,freeze_weights=True,wabdb=False,dropout=False,z_and_zz_to_y=True)

In [7]:
configs[0]['beta']

{'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}

In [8]:
for i in range(10):
    mod,r,lt,ltt=train_VAE(X_train_scaled,X_test_scaled,X_test,y_train.values,y_test.values,configs[0],'./Test',save_pictures=True,freeze_weights=False,partial=-1,offset=200,z_and_zz_to_y=False)
    print(ltt)

{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 200}
False


KeyboardInterrupt: 

# No Freez and z and zz to y 

In [9]:
from sklearn.model_selection import KFold
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
results = pd.DataFrame()

In [13]:
df_list = []
for conf in configs:
    r2s=[]
    r2s_test=[]
    losses_train=[]
    losses_test=[]
    for train_index, test_index in kf.split(X_train_scaled):
        X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_val_cv = y_train.to_numpy()[train_index], y_train.to_numpy()[test_index]
        model,r2,loss_train,loss_test=train_VAE(X_train_cv,X_val_cv,X_val_cv,y_train_cv,y_val_cv,conf,'./Test',save_pictures=False,freeze_weights=False)
        r2s.append(r2)
        losses_train.append(loss_train)
        losses_test.append(loss_test)
        test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.values))
        y_true = np.array([y.numpy() for _, y in test_ds],dtype=object)
        _,_,_,_,_,outputs = model(torch.Tensor(X_test_scaled).to(device))
        y_pred = outputs.cpu().detach().numpy()
        r2_test = r2_score(y_true, y_pred)
        r2s_test.append(r2_test)
        print("R^2 Test score:", r2_test)
    # Adding each config, mean R2 score, mean loss_train, and mean loss_test to the dictionary
    df_list.append({'config': conf, 
                    'mean_r2': np.mean(r2s),
                    'mean_r2_test': np.mean(r2s_test),
                    'mean_loss_train': np.mean(losses_train),
                    'mean_loss_test': np.mean(losses_test)})

# Creating DataFrame from list of dictionaries
df = pd.DataFrame(df_list)


{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 200}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8                    [-1, 3]              33
            Linear-9                   [-1, 10]          

R^2 score: 0.7344459190891569
R^2 Test score: 0.628216033946944
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 200}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8                    [-1, 3]          

Epoch 201, loss: 2288.0947265625
Epoch 401, loss: 1361.11669921875
R^2 score: 0.7959564369903759
R^2 Test score: 0.5897804036117362
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]      

Epoch 801, loss: 1800.016357421875
R^2 score: 0.8170441791376322
R^2 Test score: 0.6241403702129049
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8

Epoch 201, loss: 1960.827880859375
Epoch 401, loss: 1310.92138671875
Epoch 601, loss: 1215.85986328125
Epoch 801, loss: 1161.09423828125
R^2 score: 0.7188485240518558
R^2 Test score: 0.5573892551378989
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10

...

Epoch 201, loss: 1027.782470703125
Epoch 401, loss: 763.0899658203125
Epoch 601, loss: 896.4031982421875
Epoch 801, loss: 614.5389404296875
R^2 score: 0.692731170488747
R^2 Test score: 0.5107672574648281
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 

R^2 score: 0.678194314298016
R^2 Test score: 0.6026597193233766
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 200}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]             260
            Linear-8                    [-1, 4]          

Epoch 201, loss: 1394.1746826171875
Epoch 401, loss: 1044.288330078125
R^2 score: 0.7415711722852327
R^2 Test score: 0.5794039206618936
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]  

Epoch 201, loss: 1076.9168701171875
Epoch 401, loss: 953.7874755859375
Epoch 601, loss: 965.576171875
Epoch 801, loss: 955.149658203125
R^2 score: 0.722474656620197
R^2 Test score: 0.5409082632775917
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64] 

Epoch 201, loss: 1365.948486328125
Epoch 401, loss: 1011.633056640625
Epoch 601, loss: 839.7928466796875
Epoch 801, loss: 886.9024658203125
R^2 score: 0.6034699519831771
R^2 Test score: 0.5399690851073439
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1,

In [18]:
df

,config,mean_r2,mean_r2_test,mean_loss_train,mean_loss_test
0,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.751138,0.652951,0.063640,0.071394
1,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.740616,0.624048,0.058514,0.073432
2,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.761627,0.595864,0.055547,0.069094
3,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.755717,0.629886,0.066662,0.069157
4,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.765098,0.655522,0.061647,0.068371
5,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.746068,0.643483,0.060374,0.071029
6,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.740940,0.633814,0.052799,0.071550
7,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.689509,0.531868,0.041053,0.078156
8,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.659498,0.464296,0.030689,0.083377
9,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.755725,0.613642,0.056031,0.070196


In [21]:
df.iloc[4]

config             {'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...
mean_r2                                                     0.765098
mean_r2_test                                                0.655522
mean_loss_train                                             0.061647
mean_loss_test                                              0.068371
Name: 4, dtype: object

In [20]:
df.iloc[4].config

{'config': {'x_dim': 6,
  'z_dim': 3,
  'zz_dim': 1,
  'encoder_z_layers': [10, 10, 10],
  'encoder_zz_layers': [10, 10],
  'decoder_layers': [10, 10, 10, 6],
  'fc_z_to_y_layers': [12, 12, 1]},
 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1},
 'lr': 0.001,
 'batch_size': 32,
 'num_epochs': 600}

In [22]:
from sklearn.model_selection import KFold
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
results = pd.DataFrame()
df_list = []
for conf in configs:
    r2s=[]
    r2s_test=[]
    losses_train=[]
    losses_test=[]
    for train_index, test_index in kf.split(X_train_scaled):
        X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_val_cv = y_train.to_numpy()[train_index], y_train.to_numpy()[test_index]
        model,r2,loss_train,loss_test=train_VAE(X_train_cv,X_val_cv,X_val_cv,y_train_cv,y_val_cv,conf,'./Test',save_pictures=False,freeze_weights=False,z_and_zz_to_y=False)
        r2s.append(r2)
        losses_train.append(loss_train)
        losses_test.append(loss_test)
        test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.values))
        y_true = np.array([y.numpy() for _, y in test_ds],dtype=object)
        _,_,_,_,_,outputs = model(torch.Tensor(X_test_scaled).to(device))
        y_pred = outputs.cpu().detach().numpy()
        r2_test = r2_score(y_true, y_pred)
        r2s_test.append(r2_test)
        print("R^2 Test score:", r2_test)
    # Adding each config, mean R2 score, mean loss_train, and mean loss_test to the dictionary
    df_list.append({'config': conf, 
                    'mean_r2': np.mean(r2s),
                    'mean_r2_test': np.mean(r2s_test),
                    'mean_loss_train': np.mean(losses_train),
                    'mean_loss_test': np.mean(losses_test)})

# Creating DataFrame from list of dictionaries
df2 = pd.DataFrame(df_list)


{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 200}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8                    [-1, 3]              33
            Linear-9                   [-1, 10]          

R^2 score: 0.6699914745199241
R^2 Test score: 0.6617426505528194
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 200}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8                    [-1, 3]         

Epoch 201, loss: 3029.02392578125
Epoch 401, loss: 2019.4210205078125
R^2 score: 0.797592987041033
R^2 Test score: 0.613541826088742
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]     

Epoch 801, loss: 1188.052978515625
R^2 score: 0.757885102646916
R^2 Test score: 0.6556752505494983
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8 

Epoch 201, loss: 3018.232177734375
Epoch 401, loss: 1948.452392578125
Epoch 601, loss: 1273.198486328125
Epoch 801, loss: 1754.207763671875
R^2 score: 0.807741604306095
R^2 Test score: 0.6410011103029589
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 

...

Epoch 201, loss: 929.6444091796875
Epoch 401, loss: 644.1497192382812
Epoch 601, loss: 735.499755859375
Epoch 801, loss: 637.18505859375
R^2 score: 0.6615282740587902
R^2 Test score: 0.653973592546812
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]

R^2 score: 0.7403612132182094
R^2 Test score: 0.677859007991926
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 200}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]             260
            Linear-8                    [-1, 4]          

Epoch 201, loss: 1167.04443359375
Epoch 401, loss: 1165.029296875
R^2 score: 0.7561680126503116
R^2 Test score: 0.5371665361356501
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]       

Epoch 201, loss: 1257.927734375
Epoch 401, loss: 1146.920166015625
Epoch 601, loss: 1069.86767578125
Epoch 801, loss: 1100.666748046875
R^2 score: 0.598279227231358
R^2 Test score: 0.5281333507805059
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64] 

Epoch 201, loss: 1430.9619140625
Epoch 401, loss: 1058.8792724609375
Epoch 601, loss: 937.4259033203125
Epoch 801, loss: 981.7697143554688
R^2 score: 0.7499856460445506
R^2 Test score: 0.4683085911619945
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 

In [23]:
df2

,config,mean_r2,mean_r2_test,mean_loss_train,mean_loss_test
0,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.739064,0.648145,0.066769,0.070182
1,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.749692,0.637810,0.061221,0.071074
2,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.756156,0.652544,0.059678,0.069223
3,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.740481,0.647731,0.067649,0.071055
4,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.754717,0.635825,0.064347,0.069333
5,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.767531,0.644775,0.063023,0.067973
6,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.752869,0.618571,0.055652,0.070253
7,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.679950,0.547018,0.041595,0.081044
8,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.665221,0.524314,0.034305,0.082627
9,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.747178,0.640636,0.059408,0.070024


In [27]:
df2.iloc[12].config

{'config': {'x_dim': 6,
  'z_dim': 4,
  'zz_dim': 1,
  'encoder_z_layers': [10, 10, 10],
  'encoder_zz_layers': [10, 10],
  'decoder_layers': [10, 10, 10, 6],
  'fc_z_to_y_layers': [12, 12, 1]},
 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1},
 'lr': 0.001,
 'batch_size': 16,
 'num_epochs': 200}

In [12]:
from sklearn.model_selection import KFold
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
results = pd.DataFrame()
df_list = []
for conf in configs:
    r2s=[]
    r2s_test=[]
    losses_train=[]
    losses_test=[]
    for train_index, test_index in kf.split(X_train_scaled):
        X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_val_cv = y_train.to_numpy()[train_index], y_train.to_numpy()[test_index]
        model,r2,loss_train,loss_test=train_VAE(X_train_cv,X_val_cv,X_val_cv,y_train_cv,y_val_cv,conf,'./Test',save_pictures=False,freeze_weights=True,partial=400,offset=200)
        r2s.append(r2)
        losses_train.append(loss_train)
        losses_test.append(loss_test)
        test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.values))
        y_true = np.array([y.numpy() for _, y in test_ds],dtype=object)
        _,_,_,_,_,outputs = model(torch.Tensor(X_test_scaled).to(device))
        y_pred = outputs.cpu().detach().numpy()
        r2_test = r2_score(y_true, y_pred)
        r2s_test.append(r2_test)
        print("R^2 Test score:", r2_test)
    # Adding each config, mean R2 score, mean loss_train, and mean loss_test to the dictionary
    df_list.append({'config': conf, 
                    'mean_r2': np.mean(r2s),
                    'mean_r2_test': np.mean(r2s_test),
                    'mean_loss_train': np.mean(losses_train),
                    'mean_loss_test': np.mean(losses_test)})

# Creating DataFrame from list of dictionaries
df3 = pd.DataFrame(df_list)


{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8                    [-1, 3]              33
            Linear-9                   [-1, 10]          

Epoch 201, loss: 1143.3931884765625
Epoch 401, loss: 5689.6162109375
R^2 score: 0.6975907491104935
R^2 Test score: 0.6305497473478756
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]    

Epoch 601, loss: 538.0575561523438
R^2 score: 0.7861691866037906
R^2 Test score: 0.6296665437361608
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 800}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8 

Epoch 601, loss: 469.8168029785156
R^2 score: 0.6814997549264536
R^2 Test score: 0.6470072976995547
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8

Epoch 201, loss: 1457.232177734375
Epoch 401, loss: 5956.63916015625
Epoch 601, loss: 721.1893310546875
Epoch 801, loss: 780.9378662109375
R^2 score: 0.7183712464926983
R^2 Test score: 0.6365362605598192
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 

...

Epoch 201, loss: 577.8984985351562
Epoch 401, loss: 8765.177734375
R^2 score: 0.763654156825998
R^2 Test score: 0.6063057031852791
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]   

Epoch 201, loss: 572.0726928710938
Epoch 401, loss: 5939.072265625
R^2 score: 0.6916930558648453
R^2 Test score: 0.6386868018403582
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 800}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]  

Epoch 201, loss: 572.8317260742188
Epoch 401, loss: 4434.400390625
Epoch 601, loss: 434.283935546875
R^2 score: 0.6720842763637372
R^2 Test score: 0.5213178729922303
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 800}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Lin

Epoch 201, loss: 571.9857788085938
Epoch 401, loss: 5067.03125
Epoch 601, loss: 479.6626892089844
Epoch 801, loss: 435.4049377441406
R^2 score: 0.7284168994456763
R^2 Test score: 0.5469784545948935
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64

Epoch 201, loss: 610.408447265625
Epoch 401, loss: 7859.70556640625
Epoch 601, loss: 520.6646728515625
Epoch 801, loss: 381.5484313964844
R^2 score: 0.7039120451929569
R^2 Test score: 0.5682103944815432
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-

In [13]:
from sklearn.model_selection import KFold
from tqdm import tqdm
kf = KFold(n_splits=5, shuffle=True)
results = pd.DataFrame()
df_list = []
for conf in configs:
    r2s=[]
    r2s_test=[]
    losses_train=[]
    losses_test=[]
    for train_index, test_index in kf.split(X_train_scaled):
        X_train_cv, X_val_cv = X_train_scaled[train_index], X_train_scaled[test_index]
        y_train_cv, y_val_cv = y_train.to_numpy()[train_index], y_train.to_numpy()[test_index]
        model,r2,loss_train,loss_test=train_VAE(X_train_cv,X_val_cv,X_val_cv,y_train_cv,y_val_cv,conf,'./Test',save_pictures=False,freeze_weights=True,partial=400,offset=200,z_and_zz_to_y=False)
        r2s.append(r2)
        losses_train.append(loss_train)
        losses_test.append(loss_test)
        test_ds = TensorDataset(torch.Tensor(X_test_scaled), torch.Tensor(y_test.values))
        y_true = np.array([y.numpy() for _, y in test_ds],dtype=object)
        _,_,_,_,_,outputs = model(torch.Tensor(X_test_scaled).to(device))
        y_pred = outputs.cpu().detach().numpy()
        r2_test = r2_score(y_true, y_pred)
        r2s_test.append(r2_test)
        print("R^2 Test score:", r2_test)
    # Adding each config, mean R2 score, mean loss_train, and mean loss_test to the dictionary
    df_list.append({'config': conf, 
                    'mean_r2': np.mean(r2s),
                    'mean_r2_test': np.mean(r2s_test),
                    'mean_loss_train': np.mean(losses_train),
                    'mean_loss_test': np.mean(losses_test)})

# Creating DataFrame from list of dictionaries
df4 = pd.DataFrame(df_list)


{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8                    [-1, 3]              33
            Linear-9                   [-1, 10]          

Epoch 201, loss: 1271.9127197265625
Epoch 401, loss: 3634.568115234375
R^2 score: 0.8210215237168432
R^2 Test score: 0.6402763304679556
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]  

Epoch 601, loss: 513.314697265625
R^2 score: 0.8022987529129448
R^2 Test score: 0.6128188422606196
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 800}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8  

Epoch 601, loss: 769.4639282226562
R^2 score: 0.7245727157914816
R^2 Test score: 0.6305884443160603
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]               0
            Linear-7                    [-1, 3]              33
            Linear-8

Epoch 201, loss: 1994.6376953125
Epoch 401, loss: 8203.1865234375
Epoch 601, loss: 744.71435546875
Epoch 801, loss: 516.451904296875
R^2 score: 0.7591270296292448
R^2 Test score: 0.6603927290751679
{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': 1, 'encoder_z_layers': [10, 10, 10], 'encoder_zz_layers': [10, 10], 'decoder_layers': [10, 10, 10, 6], 'fc_z_to_y_layers': [12, 12, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 16, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]              70
              ReLU-2                   [-1, 10]               0
            Linear-3                   [-1, 10]             110
              ReLU-4                   [-1, 10]               0
            Linear-5                   [-1, 10]             110
              ReLU-6                   [-1, 10]   

...

Epoch 201, loss: 555.6329956054688
Epoch 401, loss: 7706.73681640625
R^2 score: 0.7688242813858087
R^2 Test score: 0.6130234330483216
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 600}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]

Epoch 201, loss: 621.5386962890625
Epoch 401, loss: 6444.44921875
R^2 score: 0.75451577616046
R^2 Test score: 0.6375948027906748
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 800}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 4]     

Epoch 201, loss: 635.635986328125
Epoch 401, loss: 6453.17626953125
Epoch 601, loss: 443.9038391113281
R^2 score: 0.7582465850249736
R^2 Test score: 0.6625731147758855
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 800}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-1, 64]               0
            L

Epoch 201, loss: 586.313232421875
Epoch 401, loss: 4347.20556640625
Epoch 601, loss: 528.9098510742188
Epoch 801, loss: 544.1492309570312
R^2 score: 0.6590605041610875
R^2 Test score: 0.6520580144457457
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [-

Epoch 201, loss: 626.0001220703125
Epoch 401, loss: 6220.54443359375
Epoch 601, loss: 586.0083618164062
Epoch 801, loss: 547.5289306640625
R^2 score: 0.7450131635886921
R^2 Test score: 0.6268118206300268
{'config': {'x_dim': 6, 'z_dim': 4, 'zz_dim': 1, 'encoder_z_layers': [64, 64, 64], 'encoder_zz_layers': [10, 10], 'decoder_layers': [64, 64, 64, 6], 'fc_z_to_y_layers': [32, 32, 32, 1]}, 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1}, 'lr': 0.001, 'batch_size': 32, 'num_epochs': 1000}
False
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 64]             448
              ReLU-2                   [-1, 64]               0
            Linear-3                   [-1, 64]           4,160
              ReLU-4                   [-1, 64]               0
            Linear-5                   [-1, 64]           4,160
              ReLU-6                   [

In [14]:
df3

,config,mean_r2,mean_r2_test,mean_loss_train,mean_loss_test
0,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.749043,0.632152,0.063270,0.070617
1,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.752125,0.657074,0.063477,0.070528
2,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.753090,0.636095,0.063699,0.070339
3,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.738214,0.644814,0.065693,0.074391
4,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.730067,0.650283,0.065367,0.072274
5,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.711646,0.639035,0.064256,0.074634
6,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.734003,0.629227,0.062702,0.073719
7,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.700891,0.578724,0.052761,0.077226
8,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.663623,0.584277,0.052291,0.082093
9,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.726817,0.638879,0.059957,0.074346


NameError: name 'df3' is not defined

In [12]:
configs[27]



{'config': {'x_dim': 6,
  'z_dim': 4,
  'zz_dim': 1,
  'encoder_z_layers': [10, 10, 10],
  'encoder_zz_layers': [10, 10],
  'decoder_layers': [10, 10, 10, 6],
  'fc_z_to_y_layers': [12, 12, 1]},
 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1},
 'lr': 0.001,
 'batch_size': 32,
 'num_epochs': 600}

NameError: name 'df3' is not defined

In [15]:
df4

,config,mean_r2,mean_r2_test,mean_loss_train,mean_loss_test
0,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.760942,0.638280,0.065567,0.068741
1,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.768109,0.630276,0.064835,0.069126
2,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.762600,0.637904,0.065429,0.068887
3,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.750013,0.648528,0.066350,0.069845
4,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.744163,0.644739,0.066391,0.070565
5,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.760770,0.648856,0.066564,0.069141
6,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.714175,0.621920,0.065317,0.075525
7,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.757878,0.632721,0.064999,0.068700
8,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.728272,0.642879,0.064917,0.073355
9,"{'config': {'x_dim': 6, 'z_dim': 3, 'zz_dim': ...",0.720424,0.636158,0.068366,0.075704


In [13]:
configs[23]

{'config': {'x_dim': 6,
  'z_dim': 3,
  'zz_dim': 1,
  'encoder_z_layers': [64, 64, 64],
  'encoder_zz_layers': [10, 10],
  'decoder_layers': [64, 64, 64, 6],
  'fc_z_to_y_layers': [32, 32, 32, 1]},
 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1},
 'lr': 0.001,
 'batch_size': 32,
 'num_epochs': 1000}

In [14]:
configs[2]

{'config': {'x_dim': 6,
  'z_dim': 3,
  'zz_dim': 1,
  'encoder_z_layers': [10, 10, 10],
  'encoder_zz_layers': [10, 10],
  'decoder_layers': [10, 10, 10, 6],
  'fc_z_to_y_layers': [12, 12, 1]},
 'beta': {'b1': 10000, 'b2': 1, 'b3': 1, 'b4': 10000, 'b5': 1},
 'lr': 0.001,
 'batch_size': 16,
 'num_epochs': 1000}